# Условие задачи

<img src='answer.png' width=947, height=560>

In [3]:
import pandas as pd
import numpy as np

Прочитаем данные, которые предоставленны в задании в `data.csv`

In [4]:
df = pd.read_csv('data.csv', header=0, names=['x', 'y'])

In [5]:
df

,x,y
0,6.878546,238.295702
1,9.436524,393.053669
2,3.572694,55.666155
3,1.079596,13.537515
4,9.487563,395.101230
...,...,...
994,8.714093,366.701156
995,3.783151,60.254649
996,9.104563,380.503300
997,1.717459,32.758233


Пусть у нас есть функция $f(x)$ с неизвестными коэффициентами $a$, $b$ и $c$, а также множества значений $x$ и $y$



$$
f(x) = ((a + eps_a) sin(x) + (b + eps_b)ln(x))^2 + (c + esp_c)x^2
$$

В данной задаче случайные величины $eps$, не должны влиять на ответ, поэтому для простоты попробуем найти коэффициенты $a$, $b$ и $c$, без случайных величин.

Перепишем функцию: 
$$
f(x) = (asin(x) + bln(x))^2 + cx^2
$$



Запишем **Cреднеквадратическую ошибку** **(MSE - Mean Squared Error)** между функцией $f(x, a, b, c)$ и истинными y

$$
MSE = \sum_{i=1}^{n} (f(x_i, a, b, c) - y_i)^2,
$$
где $x_i$, $y_i$- значения из датасета

<br>

Задача: минимизировать $MSE$ по переменным $a, b, c$

$$
MSE = \sum_{i=1}^{n} (f(x_i, a, b, c) - y_i)^2  →  min_{a, b, c}
$$

<br>
<br>
Подставим $f(x)$ в нашу задачу минимизации $MSE$:

$$
MSE = \sum_{i=1}^{n} ((asin(x_i) + b ln(x_i))^2 + cx_i^2 - y_i)^2  →  min_{a, b, c}
$$

Обозначием $MSE$ за какую-то функцию $z$:

$$
MSE = z
$$

тогда найдем частные производные: $dz \over da$, $dz \over db$ и $dz \over dc$

$$
{dz \over da} = 2 * \sum_{i=1}^{n} ((asin(x_i) + b ln(x_i))^2 + cx_i^2 - y_i) * (2asin^2(x_i) + 2bsin(x_i)ln(x_i))
$$

$$
{dz \over db} = 2 * \sum_{i=1}^{n} ((asin(x_i) + b ln(x_i))^2 + cx_i^2 - y_i) * (2bln^2(x_i) + 2asin(x_i)ln(x_i))
$$

$$
{dz \over dc} = 2 * \sum_{i=1}^{n} ((asin(x_i) + b ln(x_i))^2 + cx_i^2 - y_i) * (x_i)^2
$$



Запишем градиент функции $z$:

$$
gradZ = ({dz \over da}; {dz \over db}; {dz \over dc})
$$

Опишем алгоритм градиентого спуска для функции $z$, по переменным $a$, $b$ и $c$:
1. Инициализируемся в какой-то точке $betas_{start} = (a, b, c)$
2. Считаем градиент функции $z$
3. Шагаем в сторону антиградиента (направление наискорейшего убывания), нормированного на $\gamma$ (learning_rate - скорость обучения):
    * $betas_{next} = betas_{start} - \gamma * gradZ$
    * $betas_{start} = betas_{next}$
4. Повторяем предыдущие три шага, пока не сработает критерий останова. 

В данном случае выбрал следующий критерий останова: прекращаем итерироваться, когда разность по модулю $MSE$ с предыдущими $a, b, c$ и с новыми, полученными после шага в сторону антиградиента, нормированного на $\gamma$, меньше какого-то заданного $\epsilon$ (threshold):

$$
|MSE(betas_{start}) - MSE(betas_{next})| < \epsilon
$$

Реализуем данную логику в коде.
Для этого напишем класс `GradientDescent` и опишем в нем методы, необходимые для выполнения градиентного спуска

In [293]:
class GradientDescent:
    def __init__(self, x, y, learning_rate, threshold):
        self.x = x.copy()
        self.y = y.copy()
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.beta = np.array([1, 1, 1])
        self.a = self.beta[0]
        self.b = self.beta[1]
        self.c = self.beta[2]
        self.eps_a, self.eps_b, self.eps_c = np.random.uniform(-0.001, 0.001), np.random.uniform(-0.001, 0.001), np.random.uniform(-0.001, 0.001)
        
    def func(self, x, a, b, c):
        return ((a * np.sin(x) + b * np.log(x)) ** 2 + c * x ** 2)
    
    def calculate_mse(self, a, b, c):
        mse = np.mean((self.x.apply(lambda x: self.func(x, a, b, c)) - self.y) ** 2)
        return mse
    
    def calculate_gradient(self, a, b, c):
        dz_da = 2 / 999 * (((a * np.sin(self.x) + b * np.log(self.x)) ** 2 + c * self.x ** 2 - self.y) * (a * 2 * np.sin(self.x) ** 2 + 2 * b * np.sin(self.x) * np.log(self.x))).sum()
        dz_db = 2 / 999 * (((a * np.sin(self.x) + b * np.log(self.x)) ** 2 + c * self.x ** 2 - self.y) * (b * 2 * np.log(self.x) ** 2 + 2 * a * np.sin(self.x) * np.log(self.x))).sum()
        dz_dc = 2 / 999 * (((a * np.sin(self.x) + b * np.log(self.x)) ** 2 + c * self.x ** 2 - self.y) * self.x ** 2).sum()
    
        return np.array([dz_da, dz_db, dz_dc])
    
    def gradient_descent(self):
        a, b, c = self.beta[0], self.beta[1], self.beta[2]
        previous_mse = self.calculate_mse(a, b, c)
        
        print(f"Initializing")
        print(f'Betas: {self.beta}')
        print(f'MSE: {previous_mse}')
        
        gradient = self.calculate_gradient(a, b, c)
        self.beta = self.beta - self.learning_rate * gradient
        
        a, b, c = self.beta[0] + self.eps_a, self.beta[1] + self.eps_b, self.beta[2] + self.eps_c
        next_mse = self.calculate_mse(a, b, c)
        
        print(f"[1] Iteration:")
        print(f"Gradient: {gradient}")
        print(f"Betas: {self.beta}")
        print(f"MSE: {next_mse}")
        
        n = 2
        while abs(previous_mse - next_mse) >= self.threshold:
            previous_mse = self.calculate_mse(a, b, c)
            
            gradient = self.calculate_gradient(a, b, c)
            self.beta = self.beta - self.learning_rate * gradient
            
            a, b, c = self.beta[0] + self.eps_a, self.beta[1] + self.eps_b, self.beta[2] + self.eps_c
            next_mse = self.calculate_mse(a, b, c)
            
            print(f"[{n}] Iteration:")
            print(f"Gradient: {gradient}")
            print(f"Betas: {self.beta}")
            print(f"MSE: {next_mse}")
            
            n += 1
                  
        print(f"[!!!] Answer: {self.beta}")

Путем экспериментов, подберем хорошие параметры градиентного спуска: `learning_rate`, `threshold`

In [295]:
gradient = GradientDescent(x=df.x, y=df.y, learning_rate=1e-5, threshold=1e-3)

gradient.gradient_descent()

Initializing
Betas: [1 1 1]
MSE: 23456.744453335545
[1] Iteration:
Gradient: [  -475.33815832  -2040.55852019 -12770.15780836]
Betas: [1.00475338 1.02040559 1.12770158]
MSE: 21801.649348087823
[2] Iteration:
Gradient: [  -466.41262432  -2006.08387206 -12306.12269745]
Betas: [1.00941751 1.04046642 1.25076281]
MSE: 20272.55865394485
[3] Iteration:
Gradient: [  -457.67251916  -1970.85715649 -11861.33971909]
Betas: [1.01399423 1.060175   1.3693762 ]
MSE: 18850.523724353727
[4] Iteration:
Gradient: [  -448.98674204  -1935.06523261 -11432.21857534]
Betas: [1.0184841  1.07952565 1.48369839]
MSE: 17528.135913001945
[5] Iteration:
Gradient: [  -440.36440207  -1898.80514847 -11018.22175653]
Betas: [1.02288774 1.0985137  1.59388061]
MSE: 16298.494337572913
[6] Iteration:
Gradient: [  -431.81409826  -1862.16935506 -10618.83011295]
Betas: [1.02720589 1.11713539 1.70006891]
MSE: 15155.171190439083
[7] Iteration:
Gradient: [  -423.34391102  -1825.24566103 -10233.54218163]
Betas: [1.03143932 1.1353878

[98] Iteration:
Gradient: [-107.87564658 -180.87039419 -298.64654851]
Betas: [1.21483946 1.76763503 4.37865893]
MSE: 208.68648050821352
[99] Iteration:
Gradient: [-107.33385072 -177.7239042  -286.29506295]
Betas: [1.2159128  1.76941227 4.38152188]
MSE: 207.45576096917114
[100] Iteration:
Gradient: [-106.81200785 -174.69053081 -274.40627046]
Betas: [1.21698092 1.77115917 4.38426594]
MSE: 206.30203885734397
[101] Iteration:
Gradient: [-106.30941296 -171.76631947 -262.96290068]
Betas: [1.21804402 1.77287683 4.38689557]
MSE: 205.2196493627973
[102] Iteration:
Gradient: [-105.82538458 -168.94744479 -251.94832309]
Betas: [1.21910227 1.77456631 4.38941505]
MSE: 204.20334350896246
[103] Iteration:
Gradient: [-105.35926412 -166.23020703 -241.34652364]
Betas: [1.22015586 1.77622861 4.39182852]
MSE: 203.24825771790228
[104] Iteration:
Gradient: [-104.91041511 -163.61102853 -231.14208214]
Betas: [1.22120497 1.77786472 4.39413994]
MSE: 202.34988559432534
[105] Iteration:
Gradient: [-104.47822261 -1

[205] Iteration:
Gradient: [-93.93139917 -95.45700914  25.39020871]
Betas: [1.31852765 1.89085669 4.42887213]
MSE: 174.64642611521103
[206] Iteration:
Gradient: [-93.92614528 -95.38489771  25.6041512 ]
Betas: [1.31946691 1.89181054 4.42861609]
MSE: 174.460675740801
[207] Iteration:
Gradient: [-93.92118384 -95.31476376  25.81043409]
Betas: [1.32040612 1.89276368 4.42835799]
MSE: 174.27496204987244
[208] Iteration:
Gradient: [-93.91650108 -95.24652965  26.00934582]
Betas: [1.32134528 1.89371615 4.42809789]
MSE: 174.08928381491174
[209] Iteration:
Gradient: [-93.91208374 -95.18012062  26.20116394]
Betas: [1.3222844  1.89466795 4.42783588]
MSE: 173.90363992380128
[210] Iteration:
Gradient: [-93.90791907 -95.11546469  26.38615553]
Betas: [1.32322348 1.8956191  4.42757202]
MSE: 173.7180293713336
[211] Iteration:
Gradient: [-93.90399477 -95.05249255  26.56457754]
Betas: [1.32416252 1.89656963 4.42730637]
MSE: 173.53245125135325
[212] Iteration:
Gradient: [-93.900299   -94.99113748  26.7366772

[312] Iteration:
Gradient: [-93.72785399 -91.64981541  31.75901174]
Betas: [1.4189281  1.99061881 4.3966918 ]
MSE: 154.95344674106082
[313] Iteration:
Gradient: [-93.72399626 -91.62145432  31.77076205]
Betas: [1.41986534 1.99153502 4.39637409]
MSE: 154.77157915175508
[314] Iteration:
Gradient: [-93.72006635 -91.59302846  31.78234764]
Betas: [1.42080254 1.99245095 4.39605627]
MSE: 154.58976373095302
[315] Iteration:
Gradient: [-93.71606404 -91.56453657  31.7937731 ]
Betas: [1.4217397  1.9933666  4.39573833]
MSE: 154.40800081788458
[316] Iteration:
Gradient: [-93.7119891  -91.53597749  31.80504278]
Betas: [1.42267682 1.99428196 4.39542028]
MSE: 154.22629075155646
[317] Iteration:
Gradient: [-93.70784133 -91.50735007  31.8161609 ]
Betas: [1.4236139  1.99519703 4.39510212]
MSE: 154.04463387074668
[318] Iteration:
Gradient: [-93.7036205  -91.47865322  31.82713152]
Betas: [1.42455094 1.99611182 4.39478385]
MSE: 153.8630305139988
[319] Iteration:
Gradient: [-93.69932644 -91.4498859   31.83795

[423] Iteration:
Gradient: [-92.83779541 -87.99225834  32.51640518]
Betas: [1.5225547  2.09040205 4.36093701]
MSE: 135.15801249263203
[424] Iteration:
Gradient: [-92.82549399 -87.95433563  32.51992041]
Betas: [1.52348296 2.09128159 4.36061181]
MSE: 134.98393354743573
[425] Iteration:
Gradient: [-92.81311671 -87.91632537  32.52338644]
Betas: [1.52441109 2.09216075 4.36028658]
MSE: 134.80994224732365
[426] Iteration:
Gradient: [-92.8006636  -87.87822765  32.52680329]
Betas: [1.52533909 2.09303954 4.35996131]
MSE: 134.63603888637755
[427] Iteration:
Gradient: [-92.78813472 -87.84004258  32.53017103]
Betas: [1.52626697 2.09391794 4.35963601]
MSE: 134.46222375814128
[428] Iteration:
Gradient: [-92.77553009 -87.80177029  32.53348968]
Betas: [1.52719473 2.09479595 4.35931067]
MSE: 134.288497155616
[429] Iteration:
Gradient: [-92.76284975 -87.76341087  32.53675929]
Betas: [1.52812236 2.09567359 4.35898531]
MSE: 134.11485937126065
[430] Iteration:
Gradient: [-92.75009374 -87.72496445  32.539979

...

[3517] Iteration:
Gradient: [-12.29396461   3.30099465  -0.26519267]
Betas: [2.89311144 2.83109569 3.97394601]
MSE: 1.6780106506857302
[3518] Iteration:
Gradient: [-12.28647076   3.30196597  -0.26634674]
Betas: [2.8932343  2.83106267 3.97394867]
MSE: 1.6763917803374186
[3519] Iteration:
Gradient: [-12.27898308   3.30293313  -0.26749835]
Betas: [2.89335709 2.83102964 3.97395135]
MSE: 1.6747746787451636
[3520] Iteration:
Gradient: [-12.27150157   3.30389614  -0.26864752]
Betas: [2.89347981 2.83099661 3.97395403]
MSE: 1.6731593435170318
[3521] Iteration:
Gradient: [-12.26402622   3.304855    -0.26979423]
Betas: [2.89360245 2.83096356 3.97395673]
MSE: 1.671545772265379
[3522] Iteration:
Gradient: [-12.25655703   3.30580973  -0.2709385 ]
Betas: [2.89372501 2.8309305  3.97395944]
MSE: 1.669933962606768
[3523] Iteration:
Gradient: [-12.24909399   3.30676032  -0.27208033]
Betas: [2.8938475  2.83089743 3.97396216]
MSE: 1.6683239121620226
[3524] Iteration:
Gradient: [-12.24163709   3.30770678  -

[3623] Iteration:
Gradient: [-11.53276574   3.38218732  -0.37458919]
Betas: [2.90572999 2.82754944 3.97428788]
MSE: 1.515812273139014
[3624] Iteration:
Gradient: [-11.52589173   3.38275711  -0.37550384]
Betas: [2.90584525 2.82751561 3.97429164]
MSE: 1.5143683553860177
[3625] Iteration:
Gradient: [-11.51902325   3.38332347  -0.37641643]
Betas: [2.90596044 2.82748178 3.9742954 ]
MSE: 1.5129259747894408
[3626] Iteration:
Gradient: [-11.51216028   3.38388641  -0.37732695]
Betas: [2.90607556 2.82744794 3.97429917]
MSE: 1.5114851293595517
[3627] Iteration:
Gradient: [-11.50530283   3.38444593  -0.37823541]
Betas: [2.90619062 2.8274141  3.97430296]
MSE: 1.5100458171099624
[3628] Iteration:
Gradient: [-11.49845089   3.38500204  -0.37914181]
Betas: [2.9063056  2.82738025 3.97430675]
MSE: 1.508608036057679
[3629] Iteration:
Gradient: [-11.49160446   3.38555475  -0.38004617]
Betas: [2.90642052 2.82734639 3.97431055]
MSE: 1.5071717842230397
[3630] Iteration:
Gradient: [-11.48476352   3.38610407  -

[3730] Iteration:
Gradient: [-10.82747824   3.4249293   -0.46138781]
Betas: [2.91768385 2.82390424 3.9747375 ]
MSE: 1.3696519875898874
[3731] Iteration:
Gradient: [-10.8211642    3.42516652  -0.46209969]
Betas: [2.91779206 2.82386999 3.97474212]
MSE: 1.3683618943307234
[3732] Iteration:
Gradient: [-10.81485511   3.42540094  -0.46280983]
Betas: [2.91790021 2.82383574 3.97474675]
MSE: 1.3670731430636336
[3733] Iteration:
Gradient: [-10.80855096   3.42563257  -0.46351826]
Betas: [2.9180083  2.82380148 3.97475138]
MSE: 1.3657857321212328
[3734] Iteration:
Gradient: [-10.80225174   3.4258614   -0.46422495]
Betas: [2.91811632 2.82376722 3.97475603]
MSE: 1.3644996598388082
[3735] Iteration:
Gradient: [-10.79595746   3.42608746  -0.46492993]
Betas: [2.91822428 2.82373296 3.97476067]
MSE: 1.3632149245543275
[3736] Iteration:
Gradient: [-10.7896681    3.42631073  -0.46563319]
Betas: [2.91833218 2.8236987  3.97476533]
MSE: 1.3619315246084223
[3737] Iteration:
Gradient: [-10.78338367   3.42653123 

[3836] Iteration:
Gradient: [-10.18476647   3.43549082  -0.52776351]
Betas: [2.92881247 2.82026566 3.97526365]
MSE: 1.2400613955050706
[3837] Iteration:
Gradient: [-10.1789496    3.43545963  -0.52830746]
Betas: [2.92891426 2.8202313  3.97526894]
MSE: 1.2389047654050127
[3838] Iteration:
Gradient: [-10.17313717   3.43542617  -0.52884996]
Betas: [2.92901599 2.82019695 3.97527423]
MSE: 1.2377493144673046
[3839] Iteration:
Gradient: [-10.16732917   3.43539044  -0.52939103]
Betas: [2.92911767 2.82016259 3.97527952]
MSE: 1.236595041280282
[3840] Iteration:
Gradient: [-10.1615256    3.43535245  -0.52993066]
Betas: [2.92921928 2.82012824 3.97528482]
MSE: 1.2354419444344331
[3841] Iteration:
Gradient: [-10.15572645   3.43531219  -0.53046885]
Betas: [2.92932084 2.82009389 3.97529012]
MSE: 1.2342900225223996
[3842] Iteration:
Gradient: [-10.14993173   3.43526968  -0.5310056 ]
Betas: [2.92942234 2.82005953 3.97529543]
MSE: 1.2331392741389757
[3843] Iteration:
Gradient: [-10.14414143   3.43522492  

[3943] Iteration:
Gradient: [-9.58667116  3.42012341 -0.57825025]
Betas: [2.9393829  2.81659589 3.97585697]
MSE: 1.1227195174748665
[3944] Iteration:
Gradient: [-9.58130482  3.41987329 -0.57865303]
Betas: [2.93947872 2.81656169 3.97586276]
MSE: 1.1216814597667144
[3945] Iteration:
Gradient: [-9.57594248  3.41962134 -0.5790546 ]
Betas: [2.93957448 2.81652749 3.97586855]
MSE: 1.120644441618654
[3946] Iteration:
Gradient: [-9.57058411  3.41936757 -0.57945498]
Betas: [2.93967018 2.8164933  3.97587434]
MSE: 1.1196084618276474
[3947] Iteration:
Gradient: [-9.56522973  3.41911198 -0.57985417]
Betas: [2.93976583 2.81645911 3.97588014]
MSE: 1.118573519192414
[3948] Iteration:
Gradient: [-9.55987932  3.41885458 -0.58025216]
Betas: [2.93986143 2.81642492 3.97588594]
MSE: 1.1175396125134294
[3949] Iteration:
Gradient: [-9.55453288  3.41859536 -0.58064896]
Betas: [2.93995698 2.81639073 3.97589175]
MSE: 1.1165067405929134
[3950] Iteration:
Gradient: [-9.54919041  3.41833433 -0.58104457]
Betas: [2.94

In [294]:
gradient = GradientDescent(x=df.x, y=df.y, learning_rate=1e-5, threshold=1e-7)

gradient.gradient_descent()

Initializing
Betas: [1 1 1]
MSE: 23456.744453335545
[1] Iteration:
Gradient: [  -475.33815832  -2040.55852019 -12770.15780836]
Betas: [1.00475338 1.02040559 1.12770158]
MSE: 21821.900115447246
[2] Iteration:
Gradient: [  -466.78953436  -2006.46609071 -12311.93074239]
Betas: [1.00942128 1.04047025 1.25082089]
MSE: 20291.386190709552
[3] Iteration:
Gradient: [  -458.03871003  -1971.25163606 -11866.94382599]
Betas: [1.01400166 1.06018276 1.36949032]
MSE: 18868.026306873213
[4] Iteration:
Gradient: [  -449.34240397  -1935.47061248 -11437.62572784]
Betas: [1.01849509 1.07953747 1.48386658]
MSE: 17544.405087346684
[5] Iteration:
Gradient: [  -440.70972334  -1899.22012413 -11023.43869832]
Betas: [1.02290219 1.09852967 1.59410097]
MSE: 16313.615379841916
[6] Iteration:
Gradient: [  -432.1492655   -1862.59267931 -10623.86335673]
Betas: [1.02722368 1.1171556  1.7003396 ]
MSE: 15169.223535251454
[7] Iteration:
Gradient: [  -423.6691095   -1825.67614462 -10238.3980176 ]
Betas: [1.03146037 1.135412

[99] Iteration:
Gradient: [-107.30941873 -177.82235943 -286.43837068]
Betas: [1.21600579 1.76967926 4.38309076]
MSE: 207.32903284072452
[100] Iteration:
Gradient: [-106.78697083 -174.78754904 -274.5436475 ]
Betas: [1.21707366 1.77142714 4.3858362 ]
MSE: 206.1742895985852
[101] Iteration:
Gradient: [-106.28379175 -171.8619468  -263.0945677 ]
Betas: [1.2181365  1.77314576 4.38846714]
MSE: 205.09095054970498
[102] Iteration:
Gradient: [-105.79919928 -169.04172589 -252.07449264]
Betas: [1.21919449 1.77483618 4.39098789]
MSE: 204.0737614936307
[103] Iteration:
Gradient: [-105.3325341  -166.32318513 -241.46740044]
Betas: [1.22024781 1.77649941 4.39340256]
MSE: 203.11785400983553
[104] Iteration:
Gradient: [-104.88315908 -163.70274552 -231.25786336]
Betas: [1.22129664 1.77813644 4.39571514]
MSE: 202.21871721401115
[105] Iteration:
Gradient: [-104.45045861 -161.17694671 -221.43102606]
Betas: [1.22234115 1.7797482  4.39792945]
MSE: 201.3721715742437
[106] Iteration:
Gradient: [-104.03383793 -15

[205] Iteration:
Gradient: [-93.88830886 -95.5091625   25.40202341]
Betas: [1.31858041 1.89119282 4.43046508]
MSE: 174.50258215794693
[206] Iteration:
Gradient: [-93.88301158 -95.43694016  25.61605672]
Betas: [1.31951924 1.89214719 4.43020892]
MSE: 174.3168074525824
[207] Iteration:
Gradient: [-93.8780071  -95.36669629  25.82242649]
Betas: [1.32045802 1.89310085 4.42995069]
MSE: 174.1310696955168
[208] Iteration:
Gradient: [-93.87328164 -95.29835321  26.02142132]
Betas: [1.32139675 1.89405384 4.42969048]
MSE: 173.94536765781578
[209] Iteration:
Gradient: [-93.86882194 -95.23183611  26.21331891]
Betas: [1.32233544 1.89500616 4.42942835]
MSE: 173.75970022605435
[210] Iteration:
Gradient: [-93.86461521 -95.16707298  26.39838646]
Betas: [1.32327409 1.89595783 4.42916436]
MSE: 173.57406639382785
[211] Iteration:
Gradient: [-93.86064914 -95.10399449  26.57688107]
Betas: [1.32421269 1.89690887 4.42889859]
MSE: 173.38846525388172
[212] Iteration:
Gradient: [-93.8569119  -95.04253388  26.749050

[312] Iteration:
Gradient: [-93.68077829 -91.69214748  31.77195533]
Betas: [1.41893256 1.99100525 4.39827066]
MSE: 154.80843370914215
[313] Iteration:
Gradient: [-93.67688428 -91.6636998   31.78369019]
Betas: [1.41986933 1.99192189 4.39795282]
MSE: 154.6265688254935
[314] Iteration:
Gradient: [-93.67291807 -91.63518734  31.79526023]
Betas: [1.42080606 1.99283824 4.39763487]
MSE: 154.44475636425267
[315] Iteration:
Gradient: [-93.66887946 -91.60660888  31.80667001]
Betas: [1.42174275 1.99375431 4.3973168 ]
MSE: 154.26299666465954
[316] Iteration:
Gradient: [-93.6647682  -91.57796322  31.81792392]
Betas: [1.4226794  1.99467009 4.39699862]
MSE: 154.08129006572918
[317] Iteration:
Gradient: [-93.66058409 -91.54924923  31.82902617]
Betas: [1.423616   1.99558558 4.39668033]
MSE: 153.89963690624634
[318] Iteration:
Gradient: [-93.65632692 -91.52046582  31.8399808 ]
Betas: [1.42455256 1.99650078 4.39636193]
MSE: 153.71803752475913
[319] Iteration:
Gradient: [-93.6519965  -91.49161195  31.85079

[421] Iteration:
Gradient: [-92.81109312 -88.10078654  32.52007033]
Betas: [1.52064877 2.08906924 4.36315315]
MSE: 135.36320029869975
[422] Iteration:
Gradient: [-92.79890675 -88.0629541   32.52366208]
Betas: [1.52157676 2.08994987 4.36282791]
MSE: 135.18897699838195
[423] Iteration:
Gradient: [-92.78664446 -88.02503391  32.52720448]
Betas: [1.52250463 2.09083012 4.36250264]
MSE: 135.01484099745326
[424] Iteration:
Gradient: [-92.77430627 -87.98702608  32.53069758]
Betas: [1.52343237 2.09170999 4.36217733]
MSE: 134.84079259087795
[425] Iteration:
Gradient: [-92.76189223 -87.94893071  32.53414143]
Betas: [1.52435999 2.09258948 4.36185199]
MSE: 134.66683207308336
[426] Iteration:
Gradient: [-92.74940238 -87.91074792  32.53753608]
Betas: [1.52528748 2.09346858 4.36152662]
MSE: 134.49295973795734
[427] Iteration:
Gradient: [-92.73683675 -87.87247781  32.54088157]
Betas: [1.52621485 2.09434731 4.36120121]
MSE: 134.31917587884777
[428] Iteration:
Gradient: [-92.72419538 -87.8341205   32.5441

...

[14243] Iteration:
Gradient: [-0.10751451  0.05956282 -0.015194  ]
Betas: [3.13802192 2.71997876 4.00041955]
MSE: 0.0009436687097003597
[14244] Iteration:
Gradient: [-0.10747     0.05953819 -0.01518772]
Betas: [3.13802299 2.71997816 4.0004197 ]
MSE: 0.0009435154887711712
[14245] Iteration:
Gradient: [-0.10742552  0.05951357 -0.01518144]
Betas: [3.13802407 2.71997757 4.00041985]
MSE: 0.000943362394628535
[14246] Iteration:
Gradient: [-0.10738105  0.05948896 -0.01517516]
Betas: [3.13802514 2.71997697 4.00042   ]
MSE: 0.0009432094271672335
[14247] Iteration:
Gradient: [-0.1073366   0.05946436 -0.01516889]
Betas: [3.13802621 2.71997638 4.00042015]
MSE: 0.0009430565862824369
[14248] Iteration:
Gradient: [-0.10729217  0.05943977 -0.01516262]
Betas: [3.13802729 2.71997579 4.00042031]
MSE: 0.0009429038718693556
[14249] Iteration:
Gradient: [-0.10724776  0.0594152  -0.01515635]
Betas: [3.13802836 2.71997519 4.00042046]
MSE: 0.000942751283823144
[14250] Iteration:
Gradient: [-0.10720337  0.05939

[14349] Iteration:
Gradient: [-0.1028984   0.05700808 -0.0145423 ]
Betas: [3.1381334 2.719917  4.0004353]
MSE: 0.0009281130241905851
[14350] Iteration:
Gradient: [-0.10285581  0.05698451 -0.01453628]
Betas: [3.13813442 2.71991643 4.00043545]
MSE: 0.0009279726746796356
[14351] Iteration:
Gradient: [-0.10281324  0.05696095 -0.01453027]
Betas: [3.13813545 2.71991586 4.00043559]
MSE: 0.0009278324412948571
[14352] Iteration:
Gradient: [-0.10277069  0.05693739 -0.01452426]
Betas: [3.13813648 2.71991529 4.00043574]
MSE: 0.000927692323940042
[14353] Iteration:
Gradient: [-0.10272815  0.05691385 -0.01451826]
Betas: [3.13813751 2.71991472 4.00043588]
MSE: 0.0009275523225191333
[14354] Iteration:
Gradient: [-0.10268563  0.05689032 -0.01451226]
Betas: [3.13813853 2.71991415 4.00043603]
MSE: 0.0009274124369361629
[14355] Iteration:
Gradient: [-0.10264313  0.0568668  -0.01450625]
Betas: [3.13813956 2.71991358 4.00043617]
MSE: 0.0009272726670951054
[14356] Iteration:
Gradient: [-0.10260065  0.0568432

[14456] Iteration:
Gradient: [-0.09844012  0.05454047 -0.01391281]
Betas: [3.13824107 2.71985734 4.00045052]
MSE: 0.000913735494131768
[14457] Iteration:
Gradient: [-0.09839938  0.05451792 -0.01390706]
Betas: [3.13824206 2.7198568  4.00045066]
MSE: 0.0009136070402282175
[14458] Iteration:
Gradient: [-0.09835866  0.05449538 -0.01390131]
Betas: [3.13824304 2.71985625 4.0004508 ]
MSE: 0.0009134786925999917
[14459] Iteration:
Gradient: [-0.09831795  0.05447285 -0.01389556]
Betas: [3.13824402 2.71985571 4.00045094]
MSE: 0.0009133504511589554
[14460] Iteration:
Gradient: [-0.09827726  0.05445032 -0.01388982]
Betas: [3.138245   2.71985516 4.00045108]
MSE: 0.0009132223158173199
[14461] Iteration:
Gradient: [-0.09823659  0.05442781 -0.01388408]
Betas: [3.13824599 2.71985462 4.00045121]
MSE: 0.0009130942864871281
[14462] Iteration:
Gradient: [-0.09819594  0.05440531 -0.01387833]
Betas: [3.13824697 2.71985408 4.00045135]
MSE: 0.000912966363080528
[14463] Iteration:
Gradient: [-0.0981553   0.05438

[14563] Iteration:
Gradient: [-0.09417538  0.05217977 -0.01331061]
Betas: [3.13834408 2.71980027 4.00046508]
MSE: 0.0009005765165100656
[14564] Iteration:
Gradient: [-0.0941364  0.0521582 -0.0133051]
Betas: [3.13834502 2.71979975 4.00046521]
MSE: 0.0009004589491729078
[14565] Iteration:
Gradient: [-0.09409745  0.05213664 -0.0132996 ]
Betas: [3.13834596 2.71979923 4.00046534]
MSE: 0.000900341479096861
[14566] Iteration:
Gradient: [-0.09405851  0.05211508 -0.0132941 ]
Betas: [3.13834691 2.71979871 4.00046548]
MSE: 0.0009002241062012399
[14567] Iteration:
Gradient: [-0.09401959  0.05209353 -0.01328861]
Betas: [3.13834785 2.71979818 4.00046561]
MSE: 0.0009001068304056185
[14568] Iteration:
Gradient: [-0.09398068  0.052072   -0.01328311]
Betas: [3.13834879 2.71979766 4.00046574]
MSE: 0.0008999896516295864
[14569] Iteration:
Gradient: [-0.09394179  0.05205047 -0.01327762]
Betas: [3.13834972 2.71979714 4.00046588]
MSE: 0.0008998725697929487
[14570] Iteration:
Gradient: [-0.09390291  0.0520289

[14669] Iteration:
Gradient: [-0.09013302  0.049942   -0.01273976]
Betas: [3.13844173 2.71974616 4.00047888]
MSE: 0.0008886404267286255
[14670] Iteration:
Gradient: [-0.09009573  0.04992135 -0.01273449]
Betas: [3.13844263 2.71974567 4.00047901]
MSE: 0.0008885327335225804
[14671] Iteration:
Gradient: [-0.09005845  0.04990071 -0.01272923]
Betas: [3.13844353 2.71974517 4.00047913]
MSE: 0.00088842512940252
[14672] Iteration:
Gradient: [-0.09002118  0.04988008 -0.01272396]
Betas: [3.13844443 2.71974467 4.00047926]
MSE: 0.0008883176142947534
[14673] Iteration:
Gradient: [-0.08998393  0.04985946 -0.0127187 ]
Betas: [3.13844533 2.71974417 4.00047939]
MSE: 0.0008882101881255243
[14674] Iteration:
Gradient: [-0.0899467   0.04983885 -0.01271344]
Betas: [3.13844623 2.71974367 4.00047952]
MSE: 0.0008881028508210414
[14675] Iteration:
Gradient: [-0.08990948  0.04981824 -0.01270819]
Betas: [3.13844713 2.71974317 4.00047964]
MSE: 0.000887995602308046
[14676] Iteration:
Gradient: [-0.08987228  0.049797

Значения `a`, `b` и `c`, полученные во время поиска лучших параметров `learning_rate` и `threshold`

При использовании последних значений `a`, `b` и `c` (3.14, 2.72, 4.00) был достигнут наименьший MSE.

In [7]:
# best_beta = [3.07, 2.75, 4.00]
# best_beta = [3.12017791 2.7290498  3.99698554]
# best_beta = [3.13561116 2.7222317  3.99891093]
best_beta = [3.14, 2.72, 4.00] ## Answer

# Правильный ответ: [3.14, 2.72, 4.00]